# <center> Feature Reduction using general genetic algorithm </center>

### Project perspective

<center><img src="chart_perspective.jpg" ></center>

import neccessary library

In [1]:
import pandas as pd
from numpy import isnan
import numpy as np
import hazm
import itertools
from copy import deepcopy

###### Loading datasets

In [ ]:
data = pd.read_csv("Datasets/final_ds_500-500+.txt",delimiter=";")

In [3]:
data_tockenize = pd.read_csv("Datasets/final_ds_500-500+.txt",delimiter=";")

data tockenization

In [4]:
i_sample = 0
for i_sample in range(data.shape[0]):
    try:
        word = []
        Stem = hazm.Stemmer()
        temp = hazm.sent_tokenize(data_tockenize['BODY'][i_sample])
        for i_sentenses in temp :
            word.append(hazm.word_tokenize(i_sentenses))
        data_tockenize['BODY'][i_sample] = word
    except:
        pass
    try:
        word = []
        Stem = hazm.Stemmer()
        temp = hazm.sent_tokenize(data_tockenize['TITLE'][i_sample])
        for i_sentenses in temp :
            word.append(hazm.word_tokenize(i_sentenses))
        data_tockenize['TITLE'][i_sample] = word
    except:
        pass
    if(data['Sentiment'][i_sample] == 'مثبت'):
        data_tockenize['Sentiment'][i_sample] = 'Pos'
    elif(data['Sentiment'][i_sample] == 'منفی'):
        data_tockenize['Sentiment'][i_sample] = 'Neg'
    

### Convert dataset to numpyarray

In [5]:
data_temp = np.asanyarray(data_tockenize)

### Delet nan value from data set

In [6]:
del_list = []
for i_sample in range(data_temp.shape[0]):
    if(type(data_temp[i_sample,2]) != type([])):
        if(isnan(data_temp[i_sample,2])):
            del_list.append(i_sample)
data_temp = np.delete(data_temp,del_list,axis=0)

### flatening data set

In [7]:
for i_sample in range(data_temp.shape[0]):
    while(len(data_temp[i_sample,2]) > 1):
        data_temp[i_sample,2][0] += data_temp[i_sample,2][1]
        del data_temp[i_sample,2][1]
        #tmp[0] = np.ndarray.tolist(tmp[0])

In [8]:
new_data = np.zeros((0,2),dtype='object')

In [9]:
for i in range(data_temp.shape[0]):
    words = deepcopy(data_temp[i,1][0])
    words += deepcopy(data_temp[i,2][0])
    tmp = np.zeros((1,2),dtype='object')
    tmp[0,0] = data_temp[i,0]
    tmp[0,1] = words
    new_data = np.vstack((new_data,tmp))

### deleting non relative character like : + - * /

In [10]:
a = 'ا'
b = 'ی'

In [11]:
for i in range(new_data.shape[0]):
    #for each row
    for ii in range(len(new_data[i,1])):
        #for each word
        del_list = []
        for k in range(len(new_data[i,1][ii])):
            # for each character
            if(not a <= new_data[i,1][ii][k] <= b):
                del_list.append(k)
        for k in range(len(del_list)):
            # for each character
            new_data[i,1][ii] = new_data[i,1][ii].replace(new_data[i,1][ii][del_list[k]],';')
        new_data[i,1][ii] = new_data[i,1][ii].replace(';','')
        
    while '' in new_data[i,1] :
        new_data[i,1].remove('')

not that data has been cleaned we need to create word set from cleaned data

In [12]:
Wordset = set()

In [13]:
for i_sample in range(new_data.shape[0]):
    #for each row or news
    for i_word in range(len(new_data[i_sample,1])):
        #for each word in each news
        Wordset.add(new_data[i_sample,1][i_word])

In [14]:
dict_data = np.zeros((0,2),dtype='object')

calculating TF vector for each news

In [15]:
for i_sample in range(new_data.shape[0]):
    tmp = np.zeros((1,2),dtype='object')
    tmp[0,0] = new_data[i_sample,0]
    template = dict.fromkeys(Wordset,0)
    for i_word in new_data[i_sample,1]:
        template[i_word] += 1
    tmp[0,1] = template
    dict_data = np.vstack((dict_data,tmp))

In [37]:
idf = dict.fromkeys(Wordset,0)
for i_sample in range(new_data.shape[0]):
    tmp = dict.fromkeys(Wordset,0)
    for i_word in new_data[i_sample,1]:
        tmp[i_word] = 1
    idf = {k: idf.get(k, 0) + tmp.get(k, 0) for k in set(idf) }

TF IDF for each word in each news

In [44]:
idf_data = deepcopy(dict_data)

In [47]:
for i_sample in range(dict_data.shape[0]):
    for word in dict_data[i_sample,1]:
        if idf[word] < 800:
            idf_data[i_sample,1][word] = dict_data[i_sample,1][word] * ( \
                                                                        np.log(dict_data.shape[0]) - \
                                                                        np.log(idf[word]))
        else :
            idf_data[i_sample,1][word] = 0

In [61]:
for i_sample in range(dict_data.shape[0]):
    idf_data[i_sample,1].update({'class': idf_data[i_sample,0]})

### creating featured data set

In [88]:
dataset = pd.DataFrame(columns=list(idf_data[0,1]))

for i_sample in range(dict_data.shape[0]):
    tmp = pd.DataFrame([idf_data[i_sample,1]],columns=list(idf_data[0,1]))
    dataset = dataset.append(tmp)
    del tmp

In [91]:
dataset.to_csv('featured_data.csv')

In [2]:
dataset = pd.DataFrame.from_csv('featured_data.csv')

### evaluating the result

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
from sklearn.svm import SVC

In [5]:
tmp_data = np.array(dataset)

In [6]:
results = []
for i in range(10):
    train,test = train_test_split(tmp_data,test_size = 0.5)

    model = SVC()

    model.fit(train[:,0:train.shape[1]-1],train[:,train.shape[1]-1])

    results.append(float(sum(model.predict(test[:,0:test.shape[1]-1]) == test[:,test.shape[1]-1])) / test.shape[0])
print("Average result is : ",np.average(results))

Average result is :  0.811623246493


In [7]:
results

[0.8136272545090181,
 0.8016032064128257,
 0.8316633266533067,
 0.8316633266533067,
 0.8156312625250501,
 0.8016032064128257,
 0.8216432865731463,
 0.8136272545090181,
 0.8416833667334669,
 0.7434869739478958]